In [336]:
import numpy as np
import pandas as pd
from sklearn import preprocessing as prep
from sklearn.base import TransformerMixin
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import LinearSVC as LSVC
from sklearn.ensemble import RandomForestClassifier as rFor
from sklearn import cross_validation as cv
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler as sScaler

import matplotlib.pyplot as plt
%matplotlib inline

## Read in the data

In [368]:
data = pd.read_csv('data/train.csv')
 
data.select_dtypes(include=['object']).keys()
#pd.DataFrame(data.select_dtypes(include=['object'])).describe()

Index([u'v3', u'v22', u'v24', u'v30', u'v31', u'v47', u'v52', u'v56', u'v66',
       u'v71', u'v74', u'v75', u'v79', u'v91', u'v107', u'v110', u'v112',
       u'v113', u'v125'],
      dtype='object')

In [338]:
test = pd.read_csv('data/test.csv')

pd.DataFrame(test.select_dtypes(include=['object'])).describe()

,v3,v22,v24,v30,v31,v47,v52,v56,v66,v71,v74,v75,v79,v91,v107,v110,v112,v113,v125
count,110901,113916,114393,54051,110901,114393,114391,107475,114393,114393,114393,114393,114393,114391,114391,114393,113980,59035,114309
unique,3,18252,5,7,3,9,12,116,3,9,3,4,17,7,7,3,22,36,90
top,C,AGDF,E,C,A,C,J,BW,A,F,B,D,C,A,E,A,F,G,BM
freq,110640,2391,55179,32326,88590,55428,11349,11561,70375,75295,113668,75288,34705,26947,26947,55727,21717,16208,5855


## Fill in the NaN's

In [339]:
# taken from http://stackoverflow.com/questions/25239958/impute-categorical-missing-values-in-scikit-learn
#updated for ints

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() if X[c].dtype == np.dtype('float64') else X[c].median() 
                               for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
    
dataFilled = DataFrameImputer().fit_transform(data)
testFilled = DataFrameImputer().fit_transform(test)

## Transform categorical into integers

In [340]:
for col in dataFilled:
    if dataFilled[col].dtype == 'O':
        le = prep.LabelEncoder()
        le.fit(np.append(dataFilled[col],testFilled[col]))
        #print le.classes_
        trns = le.transform(dataFilled[col])
        dataFilled.loc[:,col] = trns
        trns = le.transform(testFilled[col])
        testFilled.loc[:,col] = trns

dataFilled.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,2,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,21,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,1.630686,7.464411,2,4.145098,9.191265,2.436402,2.483921,2.301630,...,6.822439,3.549938,0.598896,6,1.672658,3.239542,1.957825,0,1.925763,1.739389
2,5,1,0.943877,5.310079,2,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,5,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,2,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,64,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,1.630686,7.464411,2,4.145098,8.742359,2.436402,2.483921,1.496569,...,6.822439,3.549938,0.919812,89,1.672658,3.239542,2.030373,0,1.925763,1.739389


In [341]:
keys = set(dataFilled.keys())

keys.remove('ID')
keys.remove('target')

X = dataFilled[list(keys)]

scaler = sScaler()
scaler.fit(X)

scX = scaler.transform(X)
y = dataFilled['target']

keys = set(testFilled.keys())
keys.remove('ID')

testX = testFilled[list(keys)]

In [313]:
X_train, X_test, y_train, y_test = train_test_split(scX, y)

lr = LR(n_jobs = 4,max_iter = 2000)
lr.fit(X_train,y_train)

lsvc = LSVC()
lsvc.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [314]:
print "Linear regression score: %0.3f" % (lr.score(X_test, y_test))
print "SVM score: %0.3f" %(lsvc.score(X_test,y_test))

Linear regression score: 0.766
SVM score: 0.762


In [315]:
print lr.predict_proba(X_test)[:,0]

[ 0.53917638  0.16436882  0.18138107 ...,  0.31999578  0.0768495
  0.06497317]


In [316]:
lsvc.predict(X_test)

array([1, 1, 1, ..., 1, 1, 1])

In [317]:
keys = set(testFilled.keys())
keys.remove('ID')

testData = testFilled[list(keys)]

scTestData = scaler.transform(testData)

output = {'ID': np.array(testFilled['ID']),'PredictedProb': lr.predict_proba(scTestData)[:,0]}

output = pd.DataFrame(output)

output.to_csv('submission.csv',index=False)

In [318]:
print lr.predict(testData).shape
print np.sum(lr.predict(testData))

(114393,)
114366


In [298]:
forest = rFor(n_estimators=20,n_jobs=4)
forest.fit(X_train,y_train)

lr = LR(n_jobs = 4,max_iter = 2000)
lr.fit(X_train,y_train)

lsvc = LSVC()
lsvc.fit(X_train,y_train)

print forest.score(X_train,y_train)
print forest.score(X_test,y_test)


0.998227198507
0.766768132676


In [319]:
scores = cv.cross_val_score(forest, X, y, cv=10)
print scores

[ 0.76952681  0.77073128  0.76303359  0.76985654  0.77213086  0.77116865
  0.76662001  0.77195591  0.77116865  0.76898181]


In [320]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.77 (+/- 0.01)


In [357]:
Xsh = X.shape

bigData = np.vstack([X,testX])
bigData = prep.scale(bigData)

pca = PCA(n_components=0.9)
pca.fit(np.array(bigData))

Xpca = pca.transform(bigData[:Xsh[0],:])
testXpca = pca.transform(bigData[Xsh[0]:,:])


In [358]:
print np.cumsum(pca.explained_variance_ratio_)

[ 0.20730646  0.32585141  0.4058608   0.44721949  0.48748306  0.52442914
  0.55504786  0.5797835   0.6037403   0.62335596  0.64140009  0.65862358
  0.67535885  0.69098426  0.70605341  0.72070842  0.73400706  0.74656035
  0.75848387  0.76957474  0.78032195  0.78980575  0.79898889  0.80796381
  0.81651991  0.82465263  0.83240786  0.84003275  0.84762863  0.85501979
  0.86236036  0.86951945  0.87648579  0.88313356  0.88973061  0.89574809
  0.90151254]


In [363]:
X_train, X_test, y_train, y_test = train_test_split(Xpca, y)

forest = rFor(n_estimators=100,n_jobs=4)
forest.fit(X_train,y_train)

lr = LR(n_jobs = 4,max_iter = 2000)
lr.fit(X_train,y_train)

lsvc = LSVC()
lsvc.fit(X_train,y_train)

print "Linear regression score: %0.3f" % (lr.score(X_test, y_test))
print "SVM score: %0.3f" %(lsvc.score(X_test,y_test))
print "Forest score: %0.3f" % (forest.score(X_test,y_test))


Linear regression score: 0.760
SVM score: 0.758
Forest score: 0.764


In [364]:
scores = cv.cross_val_score(forest, X, y, cv=30)
print scores

[ 0.78299659  0.77958541  0.78378378  0.77932301  0.78955655  0.78457098
  0.77984781  0.77801102  0.77381265  0.781947    0.78378378  0.78614537
  0.77932301  0.79060614  0.77853582  0.78404618  0.78220939  0.781947
  0.77722383  0.77355025  0.78404618  0.79028871  0.77664042  0.78136483
  0.78635171  0.78267717  0.77611549  0.77480315  0.78215223  0.7855643 ]


In [365]:
scores.mean()

0.7816936583399392